In [ ]:
#Import librerie di cui necessitiamo
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
"""Creiamo una directory in cui andremo a salvare tutte le immagini che man mano
verranno generate in fase di training del modello per valutare quanto il modello
sta funzionando bene"""
os.makedirs("results_images", exist_ok=True)

In [ ]:
"""Sfruttando la libreria argparse creiamo una sorta di namespace contenente
tutte le variabili che ci serviranno da ora in avanti"""
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument("--n_epochs", type=int, default=260, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=4, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--n_classes", type=int, default=10, help="number of classes for dataset")
parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=10, help="interval between image sampling")
opt = parser.parse_args()
print(opt)

Namespace(b1=0.5, b2=0.999, batch_size=64, channels=1, f='/root/.local/share/jupyter/runtime/kernel-649541b3-32d9-49e0-a4ac-a49a946c52a2.json', img_size=32, latent_dim=100, lr=0.0002, n_classes=10, n_cpu=4, n_epochs=260, sample_interval=10)


In [ ]:
"""Al reference cuda assegniamo True se la GPU è disponibile, altrimenti gli
assegniamo False"""
cuda = True if torch.cuda.is_available() else False

In [ ]:
"""Tale funzione come parametro riceve un modulo presente all'interno del
generatore o del discriminatore e se il modulo è di tipo Conv o sottotipo di
Conv (ad esempio Conv2d) allora vengono inizializzati i pesi di tale modulo, se
il modulo è di tipo BatchNorm2d allora vengono inizializzati i pesi e i bias di
tale modulo"""
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [ ]:
"""Modello del generatore"""
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        """Come primo parametro dell'Embedding dobbiamo specificare il numero potenziale
        di target differenti che l'Embedding può ricevere in ingresso.
        Come secondo parametro è necessario specificare il numero di componenti che ogni
        singola rappresentazione generata dall'Embedding dovrà avere.
        In questo caso come primo parametro stiamo specificando opt.n_classes in quanto
        potenzialmente, il numero di target differenti che possono essere forniti in
        ingresso all'Embedding è pari al numero di classi a cui può potenzialmente può
        appartenere ogni singola immagine del dataset.
        Come secondo parametro stiamo specificando opt.latent_dim in quanto vogliamo
        che ogni singola rappresentazione del target generata dell'Embedding abbia
        un numero di componenti pari al numero di componenti del rumore
        """
        self.label_emb = nn.Embedding(opt.n_classes, opt.latent_dim)
        self.init_size = opt.img_size // 4  # Initial size before upsampling

        """Linear in ingresso riceve il Tensor 2-D di dimensione BxN restituito dal
        modulo Embedding e in uscita restituisce un Tensor 2-D di dimensione Bx(128x(self.init_size)**2)
        """
        self.l1 = nn.Sequential(
            nn.Linear(opt.latent_dim, 128 * self.init_size ** 2)
            )
        
        """Come parametro di BatchNorm2d dobbiamo specificare il numero di
        canali (feature maps) dell'immagine che viene mandata in ingresso al
        modulo"""

        """Dal momento che ad Upsample come parametro scale_factor stiamo
        specificando 2, di fatto, Upsample riceve in ingresso una determinata
        immagine e in uscita produce un'immagine che ha una dimensione doppia
        rispetto a quella dell'immagine che ha ricevuto in ingresso"""

        """Il primo parametro di Conv2d indica il numero di canali (feature maps)
        dell'immagine che viene fornita in ingresso al modulo.
        Il secondo parametro di Conv2d indica il numero di canali (feature maps)
        dell'immagine che viene prodotta in uscita dal modulo.
        """
        
        """Il primo parametro di LeakyReLu è il cosiddetto negative_slop
        mentre il secondo se posto pari a True consente un risparmio della
        quantità di memoria allocata"""
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        """Per prima cosa viene preso il Tensor 1-D labels contenente le B classi a cui
        dovranno appartenere le B immagini generate dal generatore e viene passato ad un
        modulo Embedding. Il modulo Embedding, dunque, in ingresso riceve un Tensor 1-D
        contenente B elementi e in uscita restituisce un Tensor 2-D di dimensione BxN,
        ossia un Tensor avente B righe (tante quanti sono i target) e N colonne (tante 
        quante sono le componenti del rumore). In altre parole, l'Embedding per ogni
        singolo target che riceve in ingresso restituisce una rappresentazione compatta
        dello stesso.
        Dopodiché, al reference gen_input viene assegnato un Tensor 2-D risultante dal
        prodotto tra ogni singolo elemento del Tensor 2-D restituito dall'Embedding e il
        relativo elemento del Tensor 2-D noise
        """
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        """out è un Tensor 2-D di dimensione Bx(128x(self.init_size)**2)"""
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        """out è un Tensor 4-D di dimensione Bx128x8x8. In pratica, tale Tensor
        contiene un numero di immagini pari a B, dove ogni singola immagine ha
        128 canali e ha una risoluzione di 8x8"""
        img = self.conv_blocks(out)
        return img

In [ ]:
"""Modello del discriminatore"""
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block
        """L'* che precede discriminator_block è dovuto al meccanismo di Unpacking"""    
        self.conv_blocks = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4

        # Output layers
        """Attraverso adv_layer otteniamo la probabilità che l'immagine
        fornita in ingresso al discriminatore sia reale"""
        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size ** 2, 1),
            nn.Sigmoid()
            )
        """Attraverso aux_layer otteniamo le probabilità con cui l'immagine che
        è stata fornita in ingresso al discriminatore appartiene alle diverse
        classi a cui potenzialmente può appartenere"""
        self.aux_layer = nn.Sequential(
            nn.Linear(128 * ds_size ** 2, opt.n_classes),
            nn.Softmax()
            )

    def forward(self, img):
        out = self.conv_blocks(img)
        """out.shape[0] è pari a B, ossia al numero di elementi del minibatch"""
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)


        """validity contiene la probabilità che l'immagine fornita in ingresso
        al discriminatore sia reale
        label contiene le probabilità con cui l'immagine che è stata fornita
        in ingresso al discriminatore appartiene alle diverse classi a cui
        potenzialmente può appartenere"""
        return validity, label

In [ ]:
#Definiamo le Loss Function che ci serviranno"""
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [ ]:
"""Se la GPU è disponibile spostiamo i modelli e le Loss all'interno della GPU"""
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [ ]:
#Inizializziamo i parametri dei moduli presenti all'interno del generatore e del discriminatore
"""Supponiamo di considerare la funzione apply invocata su generator. Tale funzione
in maniera iterativa prende ogni singolo modulo presente all'interno di generator
e lo passa come parametro della funzione weights_init_normal"""
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (conv_blocks): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ad

In [ ]:
# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)


Extracting ../../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw




Extracting ../../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw




Extracting ../../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw




Extracting ../../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
#Creiamo gli ottimizzatori
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [ ]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
def sample_image(n_row, epoch):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    """Per generare z, ossia il minibatch contenente rumore casuale da mandare in ingresso
    al generatore abbiamo creato un Tensor 2-D avente un numero di righe pari a
    n_row**2 e un numero di colonne pari a 100. Dopodiché, dal momento che vogliamo
    che questo Tensor contenga sempre gli stessi valori anche se la funzione sample_image
    viene chiamata più volte abbiamo deciso di wrappare tale Tensor all'interno di Variable.
    Così facendo, anche se la funzione sample_image viene chiamata più volte, il valore di z
    rimane invariato"""
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    """Per generare labels, ossia il minibatch contenente i target da mandare in ingresso
    al generatore abbiamo creato un Tensor 1-D avente un numero di elementi pari a 100
    (numero di righe di z), dove l'elemento di indice 0 assume 0, l'elemento di indice 1 assume 1, l'elemento di indice 2 assume 2
    l'elemento di indice 9 assume 9, l'elemento di indice 10 assume 0, l'elemento di indice 11 assume 1 e così via.
    Inoltre, per far si che il valore di labels non cambi nonostante la funzione sample_image venga chiamata più volte abbiamo wrappato
    tale Tensor all'interno di Variable"""
    """Prendiamo il Tensor 2-D z e il Tensor 1-D labels e mandiamoli in ingresso al generatore"""
    gen_imgs = generator(z, labels)
    """Il numero di immmagini che vengono prodotte in uscita dal gneratore è pari a n_row**2"""
    save_image(gen_imgs.data, "results_images/Epoca%d.png" % epoch, nrow=n_row, normalize=True)
    """Tramite save_image all'interno della directory results_images che abbiamo creato
    precedentemente inseriamo una griglia costituita dalle n_row**2 immagini prodotte in uscita
    dal generatore disposte su n_row righe"""

In [ ]:
for epoch in range(opt.n_epochs):
  history_d_minibatch_accuracy = []
  history_d_minibatch_loss = []
  history_g_minibatch_loss = []
  for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0] #64

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        """valid è un Tensor 2-D avente un numero di righe pari a batch_size(64)
        e un numero di colonne pari a 1. Tale Tensor contiene tutti 1 e ci 
        serve per calcolare l'adversarial loss per le immagini reali
        """
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)
        """fake è un Tensor 2-D avente un numero di righe pari a batch_size(64)
        e un numero di colonne pari a 1. Tale Tensor contiene tutti 0 e ci 
        serve per calcolare l'adversarial loss per le immagini fake
        """

        # Configure input
        """Invocando il metodo type su un Tensor è possibile effettuare il
        casting di tale Tensor"""
        real_imgs = Variable(imgs.type(FloatTensor))
        """real_imgs è un minibatch contenente un numero di immagini reali pari
        a minibatch_size(64)"""
        labels = Variable(labels.type(LongTensor))
        """labels è un minibatch contenente i minibatch_size(64) target
        corrispondenti alle 64 immagini reali presenti nel minibatch real_imgs"""

        # -----------------
        #  Train Generator
        # -----------------
        """Resettiamo il gradiente"""
        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        """z è un minibatch contenente 64 rumori casuali, ognuno di 100 elementi"""
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        """gen_labels è un Tensor 1-D contenente 64 (batch_size) target, uno per
        ogni singolo rumore casuale presente nel minibatch z"""
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)
        """gen_imgs è un minibatch contenente le 64 immagini (batch_size)
        generate dal generatore"""
        # Loss measures generator's ability to fool the discriminator
        """A questo punto prendiamo il minibatch contenente le 64 immagini
        generate dal generatore e le mandiamo in ingresso al discriminatore"""
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))
        """Calcoliamo il gradiente della g_loss"""
        g_loss.backward()
        """Ottimizziamo i parametri del generatore"""
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        """Resettiamo il gradiente"""
        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        """gen_imgs contiene le 64 (batch_size) immagini generate dal generatore
        durante la fase di training del generatore"""

        """Utilizzando detach() nel momento in cui attuiamo la backpropagation
        per aggiornare i parametri del discriminatore quello che accade è che
        effettivamente vengono aggiornati solo ed esclusivamente i parametri del
        discriminatore e non anche quelli del generatore.
        Se non usassimo detach() nel momento in cui attuaiamo la backpropagation
        per aggiornare i parametri del discriminatore, oltre ai parametri del
        discriminatore, verrebbero aggiornati anche quelli del generatore e questo
        non va bene in quanto vogliamo aggiornare solo i parametri del
        discriminatore"""
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2
        """Calcoliamo il gradiente della loss del discriminatore"""
        d_loss.backward()
        """Ottimizziamo i parametri del discriminatore"""
        optimizer_D.step()


        #Calcoliamo l'accuratezza del discriminatore nel classificare correttamente la classe di appartenenza di ogni singola immagine (fake o reale) che riceve
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        """pred ha dimensione 128x10 in quanto è dato dalla concatenazione per riga
        di real_aux (Tensor 2-D di dimensione 64x10 contenente per ogni immagine reale fornita al discriminatore
        le probabilità con cui l'immagine appartiene ad ognuna delle 10 classi) e di
        fake_aux (Tensor 2-D di dimensione 64x10 contenente per ogni immagine fake fornita al discriminatore
        le probabilità con cui l'immagine appartiene ad ognuna delle 10 classi)"""
        target = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        """target ha dimensione 128 in quanto è dato dalla concatenazione per riga
        di labels (Tensor 1-D di dimensione 64 contenente per ogni immagine reale fornita al discriminatore il relativo target)
        e di gen_labels (Tensor 1-D di dimensione 64 contenente per ogni immagine fake fornita al discriminatore il relativo target)
        """
        d_minibatch_accuracy = np.mean(np.argmax(pred, axis=1) == target)
        history_d_minibatch_accuracy.append(d_minibatch_accuracy)
        history_d_minibatch_loss.append(d_loss.item())
        history_g_minibatch_loss.append(g_loss.item())

        if (epoch+1) % opt.sample_interval == 0:
            sample_image(n_row=10, epoch=epoch+1)

  d_epoch_accuracy = np.mean(history_d_minibatch_accuracy)
  d_epoch_loss = np.mean(history_d_minibatch_loss)
  g_epoch_loss = np.mean(history_g_minibatch_loss)
  print(f"Epoch {epoch+1}: Discriminator_Loss={d_epoch_loss:.4f}, Discriminator_Accuracy={d_epoch_accuracy*100:.4f}, Generator_Loss={g_epoch_loss:.4f}")


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 1: Discriminator_Loss=1.3400, Discriminator_Accuracy=45.3183, Generator_Loss=1.4187
Epoch 2: Discriminator_Loss=1.1012, Discriminator_Accuracy=88.5178, Generator_Loss=1.2245
Epoch 3: Discriminator_Loss=1.0850, Discriminator_Accuracy=92.8163, Generator_Loss=1.1882
Epoch 4: Discriminator_Loss=1.0842, Discriminator_Accuracy=94.0898, Generator_Loss=1.1694
Epoch 5: Discriminator_Loss=1.0874, Discriminator_Accuracy=94.8144, Generator_Loss=1.1435
Epoch 6: Discriminator_Loss=1.0905, Discriminator_Accuracy=95.3858, Generator_Loss=1.1279
Epoch 7: Discriminator_Loss=1.0924, Discriminator_Accuracy=95.9355, Generator_Loss=1.1159
Epoch 8: Discriminator_Loss=1.0926, Discriminator_Accuracy=96.1354, Generator_Loss=1.1079
Epoch 9: Discriminator_Loss=1.0928, Discriminator_Accuracy=96.4844, Generator_Loss=1.1036
Epoch 10: Discriminator_Loss=1.0913, Discriminator_Accuracy=96.6010, Generator_Loss=1.1017
Epoch 11: Discriminator_Loss=1.0927, Discriminator_Accuracy=96.8467, Generator_Loss=1.0986
Epoch 12